In [70]:
from langchain_text_splitters import HTMLHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [ ]:
url = "https://python.langchain.com/v0.1/docs/get_started/introduction"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(html_header_splits)
splits

[Document(metadata={}, page_content='Skip to main content  \nThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.  \nComponentsIntegrationsGuidesAPI Reference  \nMore  \nPeopleVersioningContributingTemplatesCookbooksTutorialsYouTube  \n💬  \nv0.1  \nLatestv0.2v0.1  \n🦜️🔗  \nLangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs  \nSearch  \nSecurity  \nGet started  \nIntroductionQuickstartInstallation  \nUse cases  \nQ&A with RAG'),
 Document(metadata={}, page_content='Use cases  \nQ&A with RAG  \nExtracting structured output  \nChatbots  \nTool use and agents  \nQuery analysis  \nQ&A over SQL + CSV  \nMore  \nExpression Language  \nGet startedRunnable interfaceAdvantages of LCELStreamingAdd message history (memory)  \nPrimitives  \nMore  \nEcosystem  \n🦜🕸️ LangGraph🦜️🏓 LangServe  \n🦜🛠️ LangSmith  \nThis is documentation for LangChain v0.1, which is no longer actively mainta

In [75]:
emb={}
for i in range(len(splits)):
    print(f"Split {i+1}:")
    values = list(splits[i].metadata.values())
    result_string = ", ".join(values)
    print(result_string)
    emb[i]=result_string+" "+splits[i].page_content
    print(result_string+" "+splits[i].page_content)
    print("\n" + "="*50 + "\n")

Split 1:

 Skip to main content  
This is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.  
ComponentsIntegrationsGuidesAPI Reference  
More  
PeopleVersioningContributingTemplatesCookbooksTutorialsYouTube  
💬  
v0.1  
Latestv0.2v0.1  
🦜️🔗  
LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs  
Search  
Security  
Get started  
IntroductionQuickstartInstallation  
Use cases  
Q&A with RAG


Split 2:

 Use cases  
Q&A with RAG  
Extracting structured output  
Chatbots  
Tool use and agents  
Query analysis  
Q&A over SQL + CSV  
More  
Expression Language  
Get startedRunnable interfaceAdvantages of LCELStreamingAdd message history (memory)  
Primitives  
More  
Ecosystem  
🦜🕸️ LangGraph🦜️🏓 LangServe  
🦜🛠️ LangSmith  
This is documentation for LangChain v0.1, which is no longer actively maintained.  
For the current stable version, see this version (Latest).  
Get started  


In [76]:
emb

{0: ' Skip to main content  \nThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.  \nComponentsIntegrationsGuidesAPI Reference  \nMore  \nPeopleVersioningContributingTemplatesCookbooksTutorialsYouTube  \n💬  \nv0.1  \nLatestv0.2v0.1  \n🦜️🔗  \nLangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs  \nSearch  \nSecurity  \nGet started  \nIntroductionQuickstartInstallation  \nUse cases  \nQ&A with RAG',
 1: ' Use cases  \nQ&A with RAG  \nExtracting structured output  \nChatbots  \nTool use and agents  \nQuery analysis  \nQ&A over SQL + CSV  \nMore  \nExpression Language  \nGet startedRunnable interfaceAdvantages of LCELStreamingAdd message history (memory)  \nPrimitives  \nMore  \nEcosystem  \n🦜🕸️ LangGraph🦜️🏓 LangServe  \n🦜🛠️ LangSmith  \nThis is documentation for LangChain v0.1, which is no longer actively maintained.  \nFor the current stable version, see this version (Late

In [77]:
strings = []
for i in range(len(emb)):
    string=Document(
                page_content=emb[i],
            )
    strings.append(string)

In [78]:
embeddings = OpenAIEmbeddings(check_embedding_ctx_length=False,  openai_api_key="sk-1234", base_url="http://localhost:8080/v1",model="text-embedding-nomic-embed-text-v1.5")


In [79]:
vectorstore = Chroma.from_documents(documents=strings, 
                                    embedding=embeddings)

In [80]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [81]:
docs = retriever.get_relevant_documents("What is the main idea of the text?")

In [82]:
docs[0].page_content

"Introduction, Guides\u200b, Conceptual guide\u200b Introductions to all the key parts of LangChain you’ll need to know! Here you'll find high level explanations of all LangChain concepts.  \nFor a deeper dive into LangGraph concepts, check out this page."

In [83]:
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [84]:
llm = ChatOpenAI(base_url="http://127.0.0.1:8080/v1",model="llama-3.2-1b-instruct", api_key="LM")

In [85]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [86]:
# Run
rag_chain.invoke("Tell me about expression language")

'Based on the provided context, the Expression Language (LCEL) is described as a declarative way to compose chains in LangChain. This means that LCEL allows users to define complex expressions using simple statements and predicates, which can be used to create more complex and dynamic logic flows.\n\nTo summarize:\n\n* LCEL is a foundation for many components of LangChain.\n* It provides a declarative way to compose chains.\n* LCEL was designed to support the simplest "prompt + LLM" chain to complex chains with no code changes.'